# Strategia dla blackjack na poziomie początkującym


In [3]:
"""
Moduł implementujący graficzny interfejs użytkownika dla gry w Blackjacka z pomocą detekcji kart.

Główne funkcjonalności:
- Detekcja kart przy użyciu modelu YOLOv8
- Obliczanie wartości kart w Blackjacku
- Implementacja podstawowej strategii gry
- Interfejs graficzny z przyciskami do sterowania grą

Wymagane biblioteki:
- tkinter: do tworzenia GUI
- cv2: do przetwarzania obrazu
- numpy: do operacji na tablicach
- pyautogui: do przechwytywania ekranu
- ultralytics: do detekcji kart przy użyciu YOLOv8
"""

import tkinter as tk
import cv2
import numpy as np
import pyautogui
from collections import Counter
from ultralytics import YOLO
from tkinter import messagebox


In [4]:
# Załadowanie modelu YOLOv8 do detekcji kart
model = YOLO(r"C:\Users\48517\Desktop\Kurs CV\Projekt\yolov8s_playing_cards.pt")

# Słownik mapujący numery klas na nazwy kart
# Format: {numer_klasy: 'nazwa_karty'}, gdzie nazwa_karty to np. '10C' (10 trefl)
card_labels = {0: '10C', 1: '10D', 2: '10H', 3: '10S',
    4: '2C', 5: '2D', 6: '2H', 7: '2S', 8: '3C', 9: '3D',
    10: '3H', 11: '3S', 12: '4C', 13: '4D', 14: '4H', 15: '4S',
    16: '5C', 17: '5D', 18: '5H', 19: '5S', 20: '6C', 21: '6D',
    22: '6H', 23: '6S', 24: '7C', 25: '7D', 26: '7H', 27: '7S',
    28: '8C', 29: '8D', 30: '8H', 31: '8S', 32: '9C', 33: '9D',
    34: '9H', 35: '9S', 36: 'AC', 37: 'AD', 38: 'AH', 39: 'AS',
    40: 'JC', 41: 'JD', 42: 'JH', 43: 'JS', 44: 'KC', 45: 'KD',
    46: 'KH', 47: 'KS', 48: 'QC', 49: 'QD', 50: 'QH', 51: 'QS'}

# Słownik wartości kart w grze Blackjack
# Karty 2-10 mają wartość nominalną
# Walet (J), Dama (Q), Król (K) mają wartość 10
# As (A) ma wartość 11 (może być zmieniona na 1 w razie potrzeby)
blackjack_values = {**{card: int(card[:-1]) for card in card_labels.values() if card[:-1].isdigit()},
    **{card: 10 for card in card_labels.values() if card[:-1] in ["J", "Q", "K"]},
    **{card: 11 for card in card_labels.values() if card[:-1] == "A"}}

# Zmienne globalne używane w procesie detekcji kart
card_detection_counter = Counter()  # Licznik detekcji dla każdej karty
CONFIDENCE_THRESHOLD = 0.8  # Próg pewności dla detekcji (0.0-1.0)
FRAME_THRESHOLD = 10  # Minimalna liczba klatek z tą samą detekcją

In [5]:
def basic_strategy(player_total, dealer_value, soft):
    """ This is a simple implementation of Blackjack's
        basic strategy. It is used to recommend actions
        for the player. """

    if 4 <= player_total <= 8:
        return 'hit'
    if player_total == 9:
        if dealer_value in [1,2,7,8,9,10]:
            return 'hit'
        return 'double'
    if player_total == 10:
        if dealer_value in [1, 10]:
            return 'hit'
        return 'double'
    if player_total == 11:
        if dealer_value == 1:
            return 'hit'
        return 'double'
    if soft:
        #we only double soft 12 because there's no splitting
        if player_total in [12, 13, 14]:
            if dealer_value in [5, 6]:
                return 'double'
            return 'hit'
        if player_total in [15, 16]:
            if dealer_value in [4, 5, 6]:
                return 'double'
            return 'hit'
        if player_total == 17:
            if dealer_value in [3, 4, 5, 6]:
                return 'double'
            return 'hit'
        if player_total == 18:
            if dealer_value in [3, 4, 5, 6]:
                return 'double'
            if dealer_value in [2, 7, 8]:
                return 'stand'
            return 'hit'
        if player_total >= 19:
            return 'stand'

    else:
        if player_total == 12:
            if dealer_value in [1, 2, 3, 7, 8, 9, 10]:
                return 'hit'
            return 'stand'
        if player_total in [13, 14, 15, 16]:
            if dealer_value in [2, 3, 4, 5, 6]:
                return 'stand'
            return 'hit'

        if player_total >= 17:
            return 'stand'

In [6]:
def capture_screen(region=None):
    """
    Przechwytuje zrzut ekranu z określonego regionu.
    
    Args:
        region (tuple, optional): Region ekranu do przechwycenia (left, top, width, height).
            Jeśli None, przechwytuje cały ekran.
    
    Returns:
        numpy.ndarray: Obraz w formacie BGR.
    """
    screenshot = pyautogui.screenshot(region=region)
    frame = np.array(screenshot)
    return cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

def detect_card(exclude=[]):
    """
    Wykrywa kartę na ekranie z wykorzystaniem modelu YOLOv8.
    
    Args:
        exclude (list): Lista kart do wykluczenia z detekcji.
    
    Returns:
        str: Nazwa wykrytej karty (np. '10C' dla 10 trefl).
    """
    global card_detection_counter
    while True:
        frame = capture_screen()
        results = model.predict(frame, conf=CONFIDENCE_THRESHOLD)
        detected_card = None
        for result in results:
            for box in result.boxes:
                if box.conf[0].item() >= CONFIDENCE_THRESHOLD:
                    class_id = int(box.cls[0].item())
                    detected_card = card_labels.get(class_id)
                    if detected_card and detected_card not in exclude:
                        card_detection_counter[detected_card] += 1
                        if card_detection_counter[detected_card] >= FRAME_THRESHOLD:
                            return detected_card

def calculate_blackjack_value(cards):
    """
    Oblicza wartość kart w grze Blackjack.
    
    Args:
        cards (list): Lista kart do obliczenia wartości.
    
    Returns:
        int: Suma wartości kart, z uwzględnieniem specjalnych zasad dla asów.
    """
    value = sum(blackjack_values[card] for card in cards)
    num_aces = sum(1 for card in cards if card.startswith("A"))
    while value > 21 and num_aces > 0:
        value -= 10
        num_aces -= 1
    return value

def start_game():
    """
    Rozpoczyna nową grę w Blackjacka.
    
    Wykrywa początkowe karty gracza i krupiera, aktualizuje wyświetlacz
    i przywraca przycisk "Hit".
    """
    global player_cards, dealer_card
    player_cards = [detect_card()]
    dealer_card = detect_card(exclude=player_cards)
    player_cards.append(detect_card(exclude=player_cards + [dealer_card]))
    update_display()
    
    # Przywrócenie przycisku "Hit" po rozpoczęciu nowej gry
    hit_button.config(bg="gray", state="normal")
    hit_button.pack(pady=10)
    
    update_display()

def hit():
    """
    Dobiera dodatkową kartę dla gracza.
    
    Wykrywa nową kartę i aktualizuje wyświetlacz.
    """
    new_card = detect_card(exclude=player_cards + [dealer_card])
    player_cards.append(new_card)
    update_display()
    
def update_display():
    """
    Aktualizuje interfejs użytkownika.
    """
    if not root.winfo_exists():  # Jeśli okno zostało zamknięte, nie aktualizujemy
        return  

    player_value = calculate_blackjack_value(player_cards)
    dealer_value = blackjack_values[dealer_card]
    soft = any(card.startswith("A") for card in player_cards)

    strategy = basic_strategy(player_value, dealer_value, soft)

    # Aktualizacja treści etykiet zamiast ich ponownego tworzenia
    cards_label.config(text=f"Your cards: {', '.join(player_cards)} (Value: {player_value})")
    dealer_label.config(text=f"Dealer's value: {dealer_value}")
    strategy_label.config(text=f"Strategy: {strategy}")

    # Ukrywanie lub pokazywanie przycisku "Hit"
    if strategy == "stand":
        hit_button.pack_forget()
    else:
        hit_button.pack(pady=10)
        hit_button.config(bg="green")  # Zmiana koloru na zielony, jeśli można dobrać kartę

        


In [11]:
# Tworzenie głównego okna aplikacji
root = tk.Tk()
root.title("Blackjack Helper")  # Ustawienie tytułu okna
root.geometry("450x300")  # Ustawienie wymiarów okna

# Utworzenie etykiety wyświetlającej karty gracza
cards_label = tk.Label(root, text="Your cards: Null", font=("Arial", 12))
cards_label.pack(pady=10)  # Dodanie etykiety do okna z odstępem 10 pikseli

# Utworzenie etykiety wyświetlającej wartość kart krupiera
dealer_label = tk.Label(root, text="Dealer's value: Null", font=("Arial", 12))
dealer_label.pack(pady=10)  # Dodanie etykiety do okna z odstępem 10 pikseli

# Utworzenie etykiety wyświetlającej sugerowaną strategię
strategy_label = tk.Label(root, text="Strategia: N/A", font=("Arial", 12), fg="blue")
strategy_label.pack(pady=10)  # Dodanie etykiety do okna z odstępem 10 pikseli

# Utworzenie przycisku rozpoczynającego nową grę
start_button = tk.Button(root, text="Start", command=start_game, font=("Arial", 12), bg="blue", fg="white")
start_button.pack(pady=10)  # Dodanie przycisku do okna z odstępem 10 pikseli

# Utworzenie przycisku do dobierania dodatkowych kart (początkowo szary)
hit_button = tk.Button(root, text="Hit", command=hit, font=("Arial", 12), bg="gray", fg="white")
hit_button.pack(pady=10)  # Dodanie przycisku do okna

# Uruchomienie głównej pętli aplikacji
root.mainloop()
